In [1]:
#pip install https://github.com/acpopescu/bitsandbytes/releases/download/v0.37.2-win.1/bitsandbytes-0.37.2-py3-none-any.whl

In [2]:
# pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
# pip install https://github.com/acpopescu/bitsandbytes/releases/download/v0.37.2-win.1/bitsandbytes-0.37.2-py3-none-any.whl
# pip install torch transformers accelerate
# pip install bitsandbytes-windows
# pip install bitsandbytes

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

# torch.cuda.is_available()
# torch.cuda.device_count()
# torch.cuda.current_device()
# torch.cuda.get_device_name(0)
# torch.cuda.get_device_name(1)

# use an A100 on Colab Pro (or any system with >30GB VRAM on your own machine) to load this in bf16
# if not available, use a GPU with minimum 20GB VRAM to load this in 8bit, or with minimum 12GB of VRAM to load in 4bit
# on Colab, works with a V100 but crashes on a T4

# downloading the model and then loading it to memory step takes around 10 minutes the first time. Be patient :)

torch.cuda.empty_cache()

c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import psutil

def clear_ram_cache():
    psutil.virtual_memory().available  # This can help free up some cached memory

# Call the function to attempt to clear RAM cache
clear_ram_cache()

In [5]:
# Import data and load it to Database

df = pd.read_csv(r"C:\Users\cemticsai\Documents\Projects\text2sql\SQL code\SQL data\Cell_level_daily.csv")

import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('cell_wise.sqlite')
c = conn.cursor()

# Create the table
c.execute('''CREATE TABLE IF NOT EXISTS cell_wise_daily (
  date DATE,
  circle VARCHAR(50) PRIMARY KEY, -- Unique ID for each circle
  airtel_circle_id VARCHAR(50),
  airtel_site_id VARCHAR(50),
  airtel_cell_id VARCHAR(50), 
  freq_kind VARCHAR(50),
  bandwidth DECIMAL(10, 2),
  cell_technology VARCHAR(50),
  vendor VARCHAR(50),
  district VARCHAR(50),
  Data_Volume_DL_24_Hr DECIMAL(10, 2),
  Data_Volume_UL_24_Hr DECIMAL(10, 2),
  Data_Drop_Call_Rate_24_Hr DECIMAL(10, 2),
  Cell_Outage_24_hr DECIMAL(10, 2))''')

# Commit the changes and close the connection
conn.commit()

df.to_sql('cell_wise_daily', conn, if_exists='replace', index = False)

60000

In [6]:
df['date'].unique()

array(['11-12-2020', '12-12-2020', '13-12-2020'], dtype=object)

In [7]:
torch.cuda.is_available()
torch.cuda.device_count()


2

In [8]:
# Fetching model

model_name = r"C:\Users\cemticsai\Documents\Projects\text2sql\SQL code\SQL project"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    #load_in_4bit=True,
    device_map="auto",
    use_cache=True,
    offload_folder="offload", torch_dtype=torch.float16
)




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.7\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: C:\Users\cemticsai\anaconda3\envs\torchgpu did not contain cudart64_110.dll as expected! Searching further paths...
  warn(msg)
c:\Users\cemticsai\anaconda3\envs\torchgpu\lib\site-packages\bitsandbytes\cuda_setup\main.py:141: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/Library/usr/bin'), WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/Library/mingw-w64/bin'), WindowsPath('C:/Users/cemticsai/anaconda3/envs/torchgpu/bin')}
  warn(msg)
Loading checkpoint shards: 100%|██████████| 4/4 [00:26<00:00,  6.63s/it]


In [27]:

import re

# Define a regular expression pattern to match yyyy-mm-dd dates
pattern = r'(\d{4})-(\d{2})-(\d{2})'

# Function to replace matched dates with dd-mm-yyyy format
def replace_date(match):
    year, month, day = match.groups()
    return f'{day}-{month}-{year}'



import json

# Load the mapping dictionary from the JSON file
with open("mapping_dict.json", "r") as json_file:
    mapping_dict = json.load(json_file)



    
def text2sql(question):
  # Sample input string
  input_string = question

  # Replace words according to the mapping dictionary
  output_string = input_string
  for key, replacement in mapping_dict.items():
      output_string = output_string.replace(key, replacement)
  question = output_string

  prompt = """### Instructions:
  Your task is convert a question into a SQL query, given a Postgres database schema.
  Adhere to these rules:
  - Deliberately go through the question and database schema word by word to appropriately answer the question
  - When creating a ratio, always cast the numerator as float
  - when dl data volumn, or data volumn  replace it with Data_Volume_DL_24_Hr
  - when ul data volumn replace it with Data_Volume_UL_24_Hr
  - when drop rate or drop replace it Data_Drop_Call_Rate_24_Hr
  - when district_level_daily table

  ### Input:
  Generate a SQL query that answers the question `{question}`.
  This query will run on a database whose schema is represented in this string:

 CREATE TABLE cell_wise_daily (
    date DATE,
    district VARCHAR(50) PRIMARY KEY, -- Unique ID for each district
    circle VARCHAR(50) CHECK (circle IN ('Delhi','Kolkata')),
    airtel_circle_id VARCHAR(50),
    airtel_site_id VARCHAR(50),
    airtel_cell_id VARCHAR(50),
    freq_kind VARCHAR(50),
    bandwidth VARCHAR(50),
    cell_technology VARCHAR(50),
    vendor VARCHAR(50),
    Data_Volume_DL_24_Hr DECIMAL(10, 2),
    Data_Volume_UL_24_Hr DECIMAL(10, 2),
    Data_Drop_Call_Rate_24_Hr DECIMAL(10, 2),
    Cell_Outage_24_hr DECIMAL(10, 2)
);



  ### Response:
  Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
  ```sql
  """.format(question=question)


      ##################################

  eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
      # excruciatingly slow on an V100 with 4bit quantization
      # takes around 1-2 minutes per query
      # on a single A100 40GB, takes ~10-20 seconds
  #torch.cuda.memory_summary(device=None, abbreviated=True)

  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
          **inputs,
          num_return_sequences=1,
          eos_token_id=eos_token_id,
          pad_token_id=eos_token_id,
          max_new_tokens=300,
          do_sample=False,
          num_beams=5
      )

  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  t= outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";"
  t1=t.replace("district_level_daily","District_level_daily").replace("        ", "\n").replace("data_drop_call_rate_24_hr","Data_Drop_Call_Rate_24_Hr").\
    replace("\n\n","\n").replace("\n\n","\n").\
      replace("\n\n","\n").replace("airtel_airtel_cell_id","airtel_cell_id").\
      replace("cell_outage_24_hr","Cell_Outage_24_hr ").replace("data_volume_dl_24_hr","Data_Volume_DL_24_Hr ")
  # t1 = re.sub(pattern, replace_date, t1)
  print("\n",question)
  return t1,outputs


In [29]:
# Run Query here

text,outputs=text2sql("show me the count cells by date")
pattern = r'\b(\d{4})-(\d{2})-(\d{2})\b'
new_text = re.sub(pattern, r'\3-\2-\1', text)
print(new_text)


 show me the count airtel_cell_ids by date
SELECT cell_wise_daily.date,
  count(cell_wise_daily.airtel_cell_id) AS cell_count
   FROM   cell_wise_daily
   WHERE  cell_wise_daily.airtel_cell_id IS NOT NULL
   GROUP BY cell_wise_daily.date
   ORDER BY cell_wise_daily.date;


In [11]:
# Get Data

c.execute('''SELECT cell_wise_daily.airtel_cell_id,
  cell_wise_daily.Data_Drop_Call_Rate_24_Hr
   FROM   cell_wise_daily
   WHERE  cell_wise_daily.date >= '11-12-2020'
      AND cell_wise_daily.date <= '13-12-2020'
      AND cell_wise_daily.district = 'Delhi'
   GROUP BY cell_wise_daily.airtel_cell_id, cell_wise_daily.date
   ORDER BY cell_wise_daily.airtel_cell_id, cell_wise_daily.date;
          
           ''')

for row in c.fetchall():
     
     
     
     


SyntaxError: unexpected EOF while parsing (17680126.py, line 18)

In [ ]:
t1=t.replace("district_level_daily","District_level_daily").replace("        ", "\n").replace("data_drop_call_rate_24_hr","Data_Drop_Call_Rate_24_Hr").\
    replace("cell_id","airtel_cell_id").replace("cell_wise_daily.district = 'Delhi'","cell_wise_daily.circle = 'Delhi'").\
    replace("cell_wise_daily.district = 'Kolkata'","cell_wise_daily.circle = 'kolkata'").replace("\n\n","\n").replace("\n\n","\n").\
      replace("\n\n","\n").replace("airtel_airtel_cell_id","airtel_cell_id").\
      replace("cell_outage_24_hr","Cell_Outage_24_hr ").replace("data_volume_dl_24_hr","Data_Volume_DL_24_Hr ")

In [ ]:
# Joining query

In [ ]:
question = "Show me the list for the cell names and their corresponding data Drop Call Rates for a given date "

SELECT cell_wise_daily.cell_name,
       District_level_daily.Data_Drop_Call_Rate_24_Hr
FROM   cell_wise_daily join District_level_daily on cell_wise_daily.district = District_level_daily.district
ORDER BY cell_wise_daily.cell_name

for row in c.fetchall():
     print (row)

SyntaxError: invalid syntax (1127831922.py, line 3)

In [ ]:
question = "Show me the list for the 4g district names and their corresponding data Drop Call Rates for a given date "
SELECT dl.district,
       dl.Data_Drop_Call_Rate_24_Hr
FROM   cell_wise_daily ccd join District_level_daily dl on ccd.district = dl.district
WHERE  ccd.cell_technology = '4G'
   and ccd.date = '2019-01-01'
ORDER BY dl.Data_Drop_Call_Rate_24_Hr DESC;

In [ ]:
question = "show me the distinct circle outage"

c.execute('''
SELECT distinct cell_wise_daily.circle,
       cell_wise_daily.cell_outage_24_hr
FROM   cell_wise_daily
ORDER BY cell_wise_daily.cell_outage_24_hr DESC;
           ''')

for row in c.fetchall():
     print (row)


In [ ]:

question = "show me total Data Volume by date"

c.execute('''
SELECT cell_wise_daily.date,
       sum(cell_wise_daily.data_volume_dl_24_hr) as total_data_volume
FROM   cell_wise_daily
GROUP BY cell_wise_daily.date
ORDER BY cell_wise_daily.date;
           ''')

for row in c.fetchall():
     print (row)

In [ ]:
question = "show me maximum bandwidth by vendor"


c.execute('''
SELECT cell_wise_daily.vendor,
       max(cell_wise_daily.bandwidth) as max_bandwidth
FROM   cell_wise_daily
GROUP BY cell_wise_daily.vendor
ORDER BY max_bandwidth desc;
           ''')

for row in c.fetchall():
     print (row)

In [ ]:
question = "Find circles with a average dl volumn  greater than 20"

c.execute('''
SELECT cell_wise_daily.circle
FROM   cell_wise_daily
GROUP BY cell_wise_daily.circle
HAVING avg(cell_wise_daily.data_volume_dl_24_hr) > 20;
           ''')

for row in c.fetchall():
     print (row)

In [ ]:
question = "Find circles with more than 10 records and an average dl data volumn greater than 50mb"


c.execute('''
SELECT cell_wise_daily.circle,
       avg(cell_wise_daily.Data_Volume_DL_24_Hr) as avg_dl_data_volumn
FROM   cell_wise_daily
GROUP BY cell_wise_daily.circle
HAVING count(*) > 10 and avg(cell_wise_daily.Data_Volume_DL_24_Hr) > 10
ORDER BY avg_dl_data_volumn desc;
           ''')

for row in c.fetchall():
     print (row)

In [ ]:
model_name = r"C:\Users\cemticsai\Documents\Projects\text2sql\SQL code\SQL project"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    # torch_dtype=torch.bfloat16,
    load_in_8bit=True,
    #load_in_4bit=True,
    device_map="auto",
    use_cache=True,
    offload_folder="offload", torch_dtype=torch.float16
)



In [ ]:
prompt = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- *Deliberately go through the question and database schema word by word* to appropriately answer the question
- When creating a ratio, always cast the numerator as float
-when dl data volumn or data volumn  replace it with Data_Volume_DL_24_Hr
-when ul data volumn replace it with Data_Volume_UL_24_Hr
-when drop rate or drop replace it Data_Drop_Call_Rate_24_Hr
-when district_level_daily table 

### Input:
Generate a SQL query that answers the question `{question}`.
This query will run on a database whose schema is represented in this string:

CREATE TABLE cell_wise_daily (
  date DATE,
  district VARCHAR(50)  PRIMARY KEY, -- Unique ID for each district
  circle VARCHAR(50) PRIMARY KEY, -- Unique ID for each circle
  airtel_circle_id VARCHAR(50) 
  airtel_site_id VARCHAR(50) 
  airtel_cell_id VARCHAR(50)
  freq_kind VARCHAR(50),
  bandwidth DECIMAL(10, 2),
  cell_technology VARCHAR(50),
  vendor VARCHAR(50));


CREATE TABLE District_level_daily (
  date DATE,
  district VARCHAR(50) PRIMARY KEY, -- Unique ID for each district
  Data_Volume_DL_24_Hr DECIMAL(10, 2),
  Data_Volume_UL_24_Hr DECIMAL(10, 2),
  Data_Drop_Call_Rate_24_Hr DECIMAL(10, 2),
  Cell_Outage_24_hr DECIMAL(10, 2)
);

CREATE TABLE Circle_level_daily (
  date DATE,
  circle VARCHAR(50) PRIMARY KEY, -- Unique ID for each circle
  Data_Volume_DL_24_Hr DECIMAL(10, 2),
  Data_Volume_UL_24_Hr DECIMAL(10, 2),
  Data_Drop_Call_Rate_24_Hr DECIMAL(10, 2),
  Cell_Outage_24_hr DECIMAL(10, 2)
);


Hint:
You can use the JOIN clause to combine data from the two tables based on the district column.
You can also use the WHERE clause to filter the results based on the freq_kind column. 
You can use the JOIN clause to combine data from the two tables based on the circle column.
You can also use the WHERE clause to filter the results based on the technology column from cell_wise_daily. 
You can use the JOIN clause to combine data from the two tables based on the cell column.
You can also use the WHERE clause to filter the results based on the freq_kind column from District_level_daily.

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
""".format(question=question)



##################################

eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]
# excruciatingly slow on an V100 with 4bit quantization
# takes around 1-2 minutes per query
# on a single A100 40GB, takes ~10-20 seconds
# torch.cuda.memory_summary(device=None, abbreviated=False)

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generated_ids = model.generate(
    **inputs,
    num_return_sequences=1,
    eos_token_id=eos_token_id,
    pad_token_id=eos_token_id,
    max_new_tokens=150,
    do_sample=False,
    num_beams=1
)

outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

t= outputs[0].split("```sql")[-1].split("```")[0].split(";")[0].strip() + ";"
t1=t.replace("district_level_daily","District_level_daily")
print(t1)
